# Dependência

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Read a specific sheet named 'Assistência Social' from an Excel file into a DataFrame.
df = pd.read_excel("Base_MUNIC_2023(1).xlsx", sheet_name='Assistência Social')

# Select a specific subset of columns from the newly created DataFrame.
df = df[['Uf', 'Cod Uf', 'CodMun', 'Desc Mun', 'MASS03','MASS04', 'MASS05', 'MASS06']]

# Add a new column named 'ano' and set its value to 2023 for all rows.
df['ano']=2023

# Rename the columns to a more standardized and readable format (snake_case).
df = df.rename(columns={'Uf':'sigla_uf',
                        'Cod Uf':'cod_uf',
                        'CodMun':'id_municipio',
                        'Desc Mun':'nome_municipio',
                        'MASS03':'genero',
                        'MASS04':'idade',
                        'MASS05':'cor_raca',
                        'MASS06':'grau_instrucao'})

# Use np.where to conditionally replace 'Pardo' with 'Parda' in the 'cor_raca' column.
df['cor_raca'] = np.where(df['cor_raca']=='Pardo','Parda',df['cor_raca'])

# Standardize various 'no data' entries to 'Sem dados' in the 'genero' column.
df['genero']=np.where(df['genero']=='Recusa','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='(**) Sem gestor','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='Não soube informar','Sem dados',df['genero'])

# Standardize various 'no data' entries to 'Sem dados' in the 'cor_raca' column.
df['cor_raca']=np.where(df['cor_raca']=='Recusa','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='Não informou','Sem dados',df['cor_raca'])
df['cor_raca']==np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='(**) Sem gestor','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='Não soube informar','Sem dados',df['cor_raca'])

# Standardize various 'no data' entries to 'Sem dados' in the 'grau_instrucao' column.
df['grau_instrucao']=np.where(df['grau_instrucao']=='Recusa','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='(**) Sem gestor','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não soube informar','Sem dados',df['grau_instrucao'])

# Replace various 'no data' entries in the 'idade' column with np.nan (Not a Number).
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**) Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não soube informar',np.nan,df['idade'])

# Convert the 'idade' column to a numeric data type, allowing for calculations.
df['idade'] =pd.to_numeric(df['idade'])

# Define the age bin edges and corresponding category labels for creating age groups.
limites = [18, 30, 50, 65, 100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

# Create a new column 'faixa_etaria' by binning the 'idade' column using pd.cut.
# 'right=False' means the intervals will be [start, end), e.g., [18, 30).
df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias, right=False)

# Reorder the columns into a final, desired sequence.
df= df[['ano', 'sigla_uf','nome_municipio','id_municipio', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

# Create a dictionary to map detailed education levels to broader, standardized categories.
dict_esco = {'Ensino fundamental incompleto':'Até Ensino Fundamental',
             'Ensino fundamental completo':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) completo':'Até Ensino Fundamental',
             'Ensino fundamental ( 1º Grau) completo':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}

# Use the dictionary to replace the values in the 'grau_instrucao' column.
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             5570 non-null   int64   
 1   sigla_uf        5570 non-null   object  
 2   nome_municipio  5570 non-null   object  
 3   id_municipio    5570 non-null   int64   
 4   genero          5570 non-null   object  
 5   cor_raca        5570 non-null   object  
 6   grau_instrucao  5570 non-null   object  
 7   faixa_etaria    5554 non-null   category
dtypes: category(1), int64(2), object(5)
memory usage: 310.4+ KB


# Código

In [ ]:
# Define the schema for the BigQuery table.
# Each bigquery.SchemaField represents a column with its name, data type, and a description.
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]

## Subindo para datalake
# Initialize the BigQuery client, specifying the Google Cloud project to use.
client = bigquery.Client(project='repositoriodedadosgpsp')
# Get a reference to the target dataset within the project.
dataset_ref = client.dataset('cargos_lideranca')

# Get a reference to the target table within the dataset.
table_ref = dataset_ref.table('MUNIC_perfil_gestor_assistencia_social_tipo_orgao_v1') # nome da tabela no padrão FONTE_algo_intuitivo_dado

# Create a job configuration object and pass the predefined schema to it.
# This ensures the table in BigQuery is created with the correct columns and data types.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start a job to load the data from the pandas DataFrame (df) into the specified BigQuery table.
# The job uses the table reference and the job configuration defined above.
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the load job to complete and raise an exception if it fails.
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=dd7fbcbd-9f0a-4388-b4c3-0716f603433e>